The main aim of this notebook is to georeference the Yelp data scraped by Erik. I will do so using the geopandas and shapely modules such that we are able to get a GeoDataFrame suitable for plotting within Python as well as conventional GIS platforms like ArcGIS and QGIS. The other reason why we want to do that is to facilitate a spatial join by census tract with the ACS data. 

_source:_
_https://gis.stackexchange.com/questions/147156/making-shapefile-from-pandas-dataframe/182234#182234_
_https://gis.stackexchange.com/questions/174159/convert-a-pandas-dataframe-to-a-geodataframe_

In [1]:
# Setting up modules
from geopandas import GeoDataFrame
import numpy as np
import pandas as pd
from shapely.geometry import Point
import matplotlib.pylab as plt
import geopy

# Data path 
path = './data/Yelp/'

In [2]:
# Importing data
MN_yelp = pd.read_csv(path + "manhattan_restaurant_data_sans_dupes.csv")
MN_yelp = pd.DataFrame(MN_yelp)
BK_yelp = pd.read_csv(path + 'brooklyn_restaurant_data_sans_dupes.csv')
BK_yelp = pd.DataFrame(BK_yelp)

In [3]:
BK_yelp.head()

,id,alias,name,is_closed,review_count,rating,price,categories,latitude,longitude,address,city,zipcode,state,country
0,6gzQLjzJk25ePm_JS7ZAug,esme-brooklyn-2,Esme,False,328,4.5,$$,newamerican|cocktailbars,40.733203,-73.954967,999 Manhattan Ave,Brooklyn,11222,NY,US
1,Swjm9no7DRqhThLlf0EHng,sama-street-brooklyn-2,Sama Street,False,58,4.5,$$,cocktailbars|panasian|tapasmallplates,40.732870,-73.954480,988 Manhattan Ave,Brooklyn,11222,NY,US
2,utM-5navObsVA5sCRHobzA,madre-brooklyn-2,Madre,False,38,5.0,MISSING,newamerican,40.733110,-73.957980,214 Franklin St,Brooklyn,11222,NY,US
3,L9SuMN3UsGipopWOe3pr9w,chiko-brooklyn-2,Chiko,False,36,5.0,MISSING,japanese|sushi,40.731900,-73.954220,954 Manhattan Ave,Brooklyn,11222,NY,US
4,vyKBwzRdNX4yiJDIFv37iw,oxomoco-brooklyn-2,Oxomoco,False,247,4.0,$$$,mexican,40.729910,-73.955480,128 Greenpoint Ave,Brooklyn,11222,NY,US


In [4]:
# Using a lambda and apply function to convert latitude and longitude cols into XY coordinates
## axis = 1 for cols
# CRS to initialize coordinate reference system

BK_yelp['geometry'] = BK_yelp.apply(lambda coord: Point((float(coord.longitude), float(coord.latitude))), axis = 1)
BK_yelp_gdf = GeoDataFrame(BK_yelp, geometry = 'geometry')
BK_yelp_gdf.crs= "+proj=longlat +ellps=NAD83 +datum=NAD83 +no_defs"


# Saving to file as .shp
## Impt to set encoding = 'utf-8' or it will throw a codec error
BK_yelp_gdf.to_file(path + "BK/" + 'BK_yelp.shp', driver='ESRI Shapefile', encoding = 'utf-8')

In [7]:
# Using a lambda and apply function to convert latitude and longitude cols into XY coordinates
## axis = 1 for cols
# CRS to initialize coordinate reference system

MN_yelp['geometry'] = MN_yelp.apply(lambda coord: Point((float(coord.longitude), float(coord.latitude))), axis = 1)
MN_yelp_gdf = GeoDataFrame(MN_yelp, geometry = 'geometry')
MN_yelp_gdf.crs= "+proj=longlat +ellps=NAD83 +datum=NAD83 +no_defs"


# Saving to file as .shp
## Impt to set encoding = 'utf-8' or it will throw a codec error
MN_yelp_gdf.to_file(path + "MN/" + 'MN_yelp.shp', driver='ESRI Shapefile', encoding = 'utf-8')

In [6]:
print(BK_yelp_gdf.head())
print(BK_yelp_gdf.shape)

                       id                   alias         name  is_closed  \
0  6gzQLjzJk25ePm_JS7ZAug         esme-brooklyn-2         Esme      False   
1  Swjm9no7DRqhThLlf0EHng  sama-street-brooklyn-2  Sama Street      False   
2  utM-5navObsVA5sCRHobzA        madre-brooklyn-2        Madre      False   
3  L9SuMN3UsGipopWOe3pr9w        chiko-brooklyn-2        Chiko      False   
4  vyKBwzRdNX4yiJDIFv37iw      oxomoco-brooklyn-2      Oxomoco      False   

   review_count  rating    price                             categories  \
0           328     4.5       $$               newamerican|cocktailbars   
1            58     4.5       $$  cocktailbars|panasian|tapasmallplates   
2            38     5.0  MISSING                            newamerican   
3            36     5.0  MISSING                         japanese|sushi   
4           247     4.0      $$$                                mexican   

    latitude  longitude             address      city zipcode state country  \
0  40.7